# 2D Toy Problems
- 2D Heat equation dense solver
- 2D Heat equation sparse solver
- 2D Advection, periodic bc
- 2D Advec-Diffusion zero bc

Ref: https://hplgit.github.io/fdm-book/doc/pub/book/pdf/fdm-book-4screen.pdf
<br>
<br>
<br>

- The Actual system???

## 2D Diffusion Equation
Looking to solve the 2D Heat equation with constant diffusion coefficient, first with FD then try convert to FV.
$$ u_t = D[u_{xx} + u_{yy}] =  \nabla \cdot \nabla Du $$
How can we do this? Following the reference above, discretise in space and produce ODE system, then step in time. We now have two dimensions, so instead of a system with $N+1$ unknowns, we will have $N_x+1 \times N_y+1)$. To account for this, we must have a mapping from $\mathbb{R}^2 \to \mathbb{R}$ so that all unknowns fit into one vector. This mapping will me given by 
$$ p = m(i,j) = j \times (N_x+1) + i$$

Discretisation in space is done using centered differences and in time will be Crank-Nicholson. Most of the faff is in actually setting up the system, discretising and solving is very similar to before (TODO: rewrite the old stuff so that it looks even more similar)

### Dense Implementation


In [3]:
import numpy as np
def solve_2d_heat_dense(initial=(lambda x,y: 1), D=1, Lx=1, Ly=1, Nx=10, Ny=10, dt=0.1, T_final=0.5, theta=0.5, U_0x=(lambda x: 0), U_0y=(lambda x: 0), U_Lx=(lambda x: 0), U_Ly=(lambda x : 0), plot =True):
    # does this setup allow for periodic BC? could maybe change so that it takes in keyword 'periodic' or 'zero'
    
    # Mesh space
    x = np.linspace(0, Lx, Nx+1)
    y = np.linspace(0, Ly, Ny+1)
    dx = x[1] - x[0]
    dy = y[1] - y[0]
    
    # Mesh time
    dt = float(dt)
    Nt = int(round(T_final/dt))
    t = np.linspace(0,Nt*dt, Nt+1)
    
    # Fourier numbers
    Fx = D*dt/dx**2  # Change when moving to finite volume
    Fy = D*dt/dy**2
    # Solution storage - note only stores two timesteps. More than that is too big! 
    #TODO: change plotting, add plotting variable
    u = np.zeros((Nx+1, Ny+1))
    u_n = np.zeros((Nx+1, Ny+1))
    
    # Creat indices in each dimension makes things more readable
    Ix = range(0, Nx+1)
    Iy = range(0, Ny+1)
    It = range(0,Nt+1)

        
    # Initial condition into u_n
    for i in Ix:
        for j in Iy:
            u_n[i,j] = initial(x[i], y[j])
#     if plot:
#         from time import sleep
#         X,Y = np.meshgrid(x, y)
#         import matplotlib.pyplot as plt
#         from mpl_toolkits.mplot3d import Axes3D
#         fig = plt.figure()
#         ax = fig.gca(projection='3d')
#         surf = ax.plot_surface(X,Y,u)
#         fig.show()        
        
    # Data structures for the linear system
    N = (Nx+1)*(Ny+1)  # no of unknowns 
    A = np.zeros((N, N))
    b = np.zeros(N)
    
    # Define our mapping from 2d -> 1d
    m = lambda i,j: j*(Nx+1) + i
    
    # Starting from bottom boundary, fill in the matrix A
    j = 0
    for i in Ix:
        p = m(i,j)
        A[p,p] = 1
    # Now loop over interior points
    for j in Iy[1:-1]: # First and last given by bdy
        i = 0
        p = m(i,j)
        A[p,p] = 1 # left bdy 
        for i in Ix[1:-1]:
            p = m(i,j)
            A[p, m(i, j-1)] = -theta*Fy
            A[p, m(i-1, j)] = -theta*Fx
            A[p,p] = 1 + 2*theta*(Fx+Fy)
            A[p, m(i+1, j)] = -theta*Fx
            A[p, m(i, j+1)] = -theta*Fy
        i = Nx
        p = m(i,j)
        A[p,p] = 1 # right bdy
    j = Ny
    for i in Ix:
        p = m(i,j)
        A[p,p] = 1 # top bdy
    
    
    # So matrix A is built. It stays constant, just need to change b at each time step.
    # Discretising in time
    import scipy.linalg
    for n in It[:-1]:
        j = 0
        for i in Ix:
            p = m(i,j)
            b[p] = U_0y(t[n+1]) # bottom bdy
        for j in Iy[1:-1]:
            i = 0 # left bdy
            p = m(i,j)
            b[p] = U_0x(t[n+1])
            for i in Ix[1:-1]:
                p = m(i,j)
                # Interior points
                x_dir = (1-theta) * (Fx*(u_n[i+1,j] - 2*u_n[i,j] + u_n[i-1,j]))
                y_dir = (1-theta) * (Fy*(u_n[i,j+1] - 2*u_n[i,j] + u_n[i,j-1]))
                # Put together
                b[p] = u_n[i,j] + x_dir + y_dir
            i = Nx
            p = m(i,j)
            b[p] = U_Lx(t[n+1]) #right bdy
        j = Ny
        for i in Ix:
            p = m(i,j)
            b[p] = U_Ly(t[n+1]) # top bdy
        
        c = scipy.linalg.solve(A,b)
        u[:,:] =  c.reshape(Ny+1,Nx+1).T

        # Update ready for next step
        u_n, u = u, u_n
#         if plot:
#             #Update surface
#             ax.collections.clear()
#             surf = ax.plot_surface(X,Y,u)
#             fig.show() #Seems no easy way for updating data
#             sleep(0.1)
    return t, u
    

Running and plotting the final timestep. change code so that it plots new surface every 10? steps. 
if n % 10: plot? Need more adaptive method if plotting many.

Or could store so many points? Then animate after.

Or just don't waste time doing it here and do it for the sparse implementation where everything will be quicker.

In [4]:
%matplotlib qt
x_len = 1
y_len = 1
x_step = 0.05
y_step = 0.05
x_points = int(x_len/x_step)
y_points = int(y_len/y_step)
diffusion = 1
T_end = 0.1
timestep = 0.01
plott = True

t, u = solve_2d_heat_dense(initial=(lambda x,y :1), D=diffusion, Lx=x_len, Ly=y_len, Nx=x_points, Ny=y_points, dt=timestep, T_final=T_end, theta=0.5, U_0x=(lambda x: 0), U_0y=(lambda x: 0), U_Lx=(lambda x: 0), U_Ly=(lambda x : 0), plot=plott)

X = np.linspace(0, x_len+x_step, x_points+1)
Y = np.linspace(0, y_len + y_step , y_points+1)
X, Y = np.meshgrid(X, Y)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.gca(projection='3d')

surf = ax.plot_surface(X,Y,u)
fig.show()

### Sparse Implementation 
speeeeeeeed

In [ ]:
import scipy.sparse
import scipy.sparse.linalg
import numpy as np
def solve_2d_heat_sparse(
    initial, a, Lx, Ly, Nx, Ny, dt, T, theta=0.5,
    U_0x=0, U_0y=0, U_Lx=0, U_Ly=0):
    """
    Full solver for the model problem using the theta-rule
    difference approximation in time. Sparse matrix with
    dedicated Gaussian elimination algorithm 
    """
    x = np.linspace(0, Lx, Nx+1)       # mesh points in x dir
    y = np.linspace(0, Ly, Ny+1)       # mesh points in y dir
    dx = x[1] - x[0]
    dy = y[1] - y[0]

    dt = float(dt)                  # avoid integer division
    Nt = int(round(T/float(dt)))
    t = np.linspace(0, Nt*dt, Nt+1) # mesh points in time

    # Mesh Fourier numbers in each direction
    Fx = a*dt/dx**2
    Fy = a*dt/dy**2
    
    u   = np.zeros((Nx+1, Ny+1))    # unknown u at new time level
    u_n = np.zeros((Nx+1, Ny+1))    # u at the previous time level

    Ix = range(0, Nx+1)
    Iy = range(0, Ny+1)
    It = range(0, Nt+1)

    # Make U_0x, U_0y, U_Lx and U_Ly functions if they are float/int
    if isinstance(U_0x, (float,int)):
        _U_0x = float(U_0x)  # Make copy of U_0x
        U_0x = lambda t: _U_0x
    if isinstance(U_0y, (float,int)):
        _U_0y = float(U_0y)  # Make copy of U_0y
        U_0y = lambda t: _U_0y
    if isinstance(U_Lx, (float,int)):
        _U_Lx = float(U_Lx)  # Make copy of U_Lx
        U_Lx = lambda t: _U_Lx
    if isinstance(U_Ly, (float,int)):
        _U_Ly = float(U_Ly)  # Make copy of U_Ly
        U_Ly = lambda t: _U_Ly

    # Load initial condition into u_n
    for i in Ix:
        for j in Iy:
            u_n[i,j] = initial(x[i], y[j])


    N = (Nx+1)*(Ny+1)
    main   = np.zeros(N)            # diagonal
    lower  = np.zeros(N-1)          # subdiagonal
    upper  = np.zeros(N-1)          # superdiagonal
    lower2 = np.zeros(N-(Nx+1))     # lower diagonal
    upper2 = np.zeros(N-(Nx+1))     # upper diagonal
    b      = np.zeros(N)            # right-hand side

    # Precompute sparse matrix
    lower_offset = 1
    lower2_offset = Nx+1

    m = lambda i, j: j*(Nx+1) + i
    j = 0; main[m(0,j):m(Nx+1,j)] = 1  # j=0 boundary line
    for j in Iy[1:-1]:             # Interior mesh lines j=1,...,Ny-1
        i = 0;   main[m(i,j)] = 1  # Boundary
        i = Nx;  main[m(i,j)] = 1  # Boundary
        # Interior i points: i=1,...,N_x-1
        lower2[m(1,j)-lower2_offset:m(Nx,j)-lower2_offset] = - theta*Fy
        lower[m(1,j)-lower_offset:m(Nx,j)-lower_offset] = - theta*Fx
        main[m(1,j):m(Nx,j)] = 1 + 2*theta*(Fx+Fy)
        upper[m(1,j):m(Nx,j)] = - theta*Fx
        upper2[m(1,j):m(Nx,j)] = - theta*Fy
    j = Ny; main[m(0,j):m(Nx+1,j)] = 1  # Boundary line

    A = scipy.sparse.diags(
        diagonals=[main, lower, upper, lower2, upper2],
        offsets=[0, -lower_offset, lower_offset,
                 -lower2_offset, lower2_offset],
        shape=(N, N), format='csc')
    #print A.todense()   # Check that A is correct

    # Time loop
    for n in It[0:-1]:

        # Compute b, vectorized version
        j = 0; b[m(0,j):m(Nx+1,j)] = U_0y(t[n+1])     # Boundary
        for j in Iy[1:-1]:
            i = 0;   p = m(i,j);  b[p] = U_0x(t[n+1]) # Boundary
            i = Nx;  p = m(i,j);  b[p] = U_Lx(t[n+1]) # Boundary
            imin = Ix[1]
            imax = Ix[-1]  # for slice, max i index is Ix[-1]-1
            b[m(imin,j):m(imax,j)] = u_n[imin:imax,j] + \
                  (1-theta)*(Fx*(
              u_n[imin+1:imax+1,j] -
            2*u_n[imin:imax,j] +
              u_n[imin-1:imax-1,j]) +
                             Fy*(
              u_n[imin:imax,j+1] -
            2*u_n[imin:imax,j] +
              u_n[imin:imax,j-1]))
        j = Ny;  b[m(0,j):m(Nx+1,j)] = U_Ly(t[n+1]) # Boundary

        # Solve matrix system A*c = b
        c = scipy.sparse.linalg.spsolve(A, b)
        # Fill u with vector c
        #for j in Iy:  # vectorize y lines
        #    u[0:Nx+1,j] = c[m(0,j):m(Nx+1,j)]
        u[:,:] = c.reshape(Ny+1,Nx+1).T

        # Update u_n before next step
        u_n, u = u, u_n

    return t,u


In [ ]:
%matplotlib qt
x_len = 1
y_len = 1
x_step = 0.005
y_step = 0.005
x_points = int(x_len/x_step)
y_points = int(y_len/y_step)
diffusion = 1
T_end = 0.01
timestep = 0.001

t, u = solve_2d_heat_sparse(initial=(lambda x,y :1), a=diffusion, Lx=x_len, Ly=y_len, Nx=x_points, Ny=y_points, dt=timestep, T=T_end, theta=0.5, U_0x=(lambda x: 0), U_0y=(lambda x: 0), U_Lx=(lambda x: 0), U_Ly=(lambda x : 0))

X = np.linspace(0, x_len+x_step, x_points+1)
Y = np.linspace(0, y_len + y_step , y_points+1)
X, Y = np.meshgrid(X, Y)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.gca(projection='3d')

surf = ax.plot_surface(X,Y,u)
fig.show()

## Advection Equation 
Use same framework, just changing $A$? 

In [ ]:
import numpy as np
def solve_2d_adv_dense(initial=(lambda x,y: 1), a=1, Lx=1, Ly=1, Nx=10, Ny=10, dt=0.1, T_final=0.5, theta=0.5, U_0x=(lambda x: 0), U_0y=(lambda x: 0), U_Lx=(lambda x: 0), U_Ly=(lambda x : 0), plot =True):
    # does this setup allow for periodic BC? could maybe change so that it takes in keyword 'periodic' or 'zero'
    
    # Mesh space
    x = np.linspace(0, Lx, Nx+1)
    y = np.linspace(0, Ly, Ny+1)
    dx = x[1] - x[0]
    dy = y[1] - y[0]
    
    # Mesh time
    dt = float(dt)
    Nt = int(round(T_final/dt))
    t = np.linspace(0,Nt*dt, Nt+1)
    
    # Wave numbers
    Cx = a*dt/dx  # Change when moving to finite volume
    Cy = a*dt/dy
    # Solution storage - note only stores two timesteps. More than that is too big! 
    #TODO: change plotting, add plotting variable
    u = np.zeros((Nx+1, Ny+1))
    u_n = np.zeros((Nx+1, Ny+1))
    
    # Creat indices in each dimension makes things more readable
    Ix = range(0, Nx+1)
    Iy = range(0, Ny+1)
    It = range(0,Nt+1)

        
    # Initial condition into u_n
    for i in Ix:
        for j in Iy:
            u_n[i,j] = initial(x[i], y[j])
#     if plot:
#         from time import sleep
#         X,Y = np.meshgrid(x, y)
#         import matplotlib.pyplot as plt
#         from mpl_toolkits.mplot3d import Axes3D
#         fig = plt.figure()
#         ax = fig.gca(projection='3d')
#         surf = ax.plot_surface(X,Y,u)
#         fig.show()        
        
    # Data structures for the linear system
    N = (Nx+1)*(Ny+1)  # no of unknowns 
    A = np.zeros((N, N))
    b = np.zeros(N)
    
    # Define our mapping from 2d -> 1d
    m = lambda i,j: j*(Nx+1) + i
    
    # Starting from bottom boundary, fill in the matrix A
    j = 0
    for i in Ix:
        p = m(i,j)
        A[p,p] = 1
    # Now loop over interior points
    for j in Iy[1:-1]: # First and last given by bdy
        i = 0
        p = m(i,j)
        A[p,p] = 1 # left bdy 
        for i in Ix[1:-1]:
            p = m(i,j)
            A[p, m(i, j-1)] = -theta*Fy
            A[p, m(i-1, j)] = -theta*Fx
            A[p,p] = 1 + 2*theta*(Fx+Fy)
            A[p, m(i+1, j)] = -theta*Fx
            A[p, m(i, j+1)] = -theta*Fy
        i = Nx
        p = m(i,j)
        A[p,p] = 1 # right bdy
    j = Ny
    for i in Ix:
        p = m(i,j)
        A[p,p] = 1 # top bdy
    
    
    # So matrix A is built. It stays constant, just need to change b at each time step.
    # Discretising in time
    import scipy.linalg
    for n in It[:-1]:
        j = 0
        for i in Ix:
            p = m(i,j)
            b[p] = U_0y(t[n+1]) # bottom bdy
        for j in Iy[1:-1]:
            i = 0 # left bdy
            p = m(i,j)
            b[p] = U_0x(t[n+1])
            for i in Ix[1:-1]:
                p = m(i,j)
                # Interior points
                x_dir = (1-theta) * (Fx*(u_n[i+1,j] - 2*u_n[i,j] + u_n[i-1,j]))
                y_dir = (1-theta) * (Fy*(u_n[i,j+1] - 2*u_n[i,j] + u_n[i,j-1]))
                # Put together
                b[p] = u_n[i,j] + x_dir + y_dir
            i = Nx
            p = m(i,j)
            b[p] = U_Lx(t[n+1]) #right bdy
        j = Ny
        for i in Ix:
            p = m(i,j)
            b[p] = U_Ly(t[n+1]) # top bdy
        
        c = scipy.linalg.solve(A,b)
        for i in Ix:
            for j in Iy:
                u[i,j] = c[m(i,j)]
        
        # Update ready for next step
        u_n, u = u, u_n
#         if plot:
#             #Update surface
#             ax.collections.clear()
#             surf = ax.plot_surface(X,Y,u)
#             fig.show() #Seems no easy way for updating data
#             sleep(0.1)
    return t, u
    